In [19]:
import numpy as np
import matplotlib.pyplot as plt
##import seaborn as sns
import pandas as pd
import pystan as stan

KeyboardInterrupt: 

In [3]:
df = pd.read_csv('MRT_Data.csv', delimiter=';',dtype={'old_new': int, 'choice': int, 'accuracy': int})
##print for debug
print(df)

    old_new  remember  accuracy
0         0         0         1
1         0         0         1
2         1         1         1
3         1         1         1
4         0         0         1
5         0         0         1
6         1         1         1
7         0         0         1
8         1         1         1
9         1         1         1
10        0         0         1
11        0         0         1
12        0         0         1
13        1         1         1
14        1         0         1
15        0         0         1
16        1         1         1
17        0         1         0
18        1         1         1
19        1         1         1


In [29]:
OneHT_program_code = """
data {
    int<lower=0> N; // Number of data points
    int<lower=0, upper=1> is_old[N]; // Indicator for old (1) or new (0) word
    int<lower=0, upper=1> is_confident[N]; // Indicator for confident (1) or not (0)
    int<lower=0, upper=1> is_correct[N]; // Indicator for correct (1) or wrong (0) response
}
parameters {
    real mu;        // Mean
    real<lower=0> sigma; // Standard deviation
    real<lower=0, upper=1> pho; // Probability of remembering a studied item
    real<lower=0, upper=1> gamma; // Probability of guessing old when no memory
    
}
model {
    //prior distributions
    pho ~ beta(1,1);
    gamma ~ beta(1,1);
    for (i in 1:N) {
        if (is_old[i] == 1&&is_correct[i]==1) {
            is_confident[i] ~ bernoulli(pho);
            if(is_confident[i]==0){1~bernoulli(gamma);} 
        }
        else{
            if(is_correct[i]==1){0~bernoulli(gamma);}
            else{1~bernoulli(gamma);}
        }
    }
}
"""

In [34]:
TwoHT_program_code = """
data {
    int<lower=0> N; // Number of data points
    int<lower=0, upper=1> is_old[N]; // Indicator for old (1) or new (0) word
    int<lower=0, upper=1> is_confident[N]; // Indicator for confident (1) or not (0)
    int<lower=0, upper=1> is_correct[N]; // Indicator for correct (1) or wrong (0) response
}
parameters {
    real mu;        // Mean
    real<lower=0> sigma; // Standard deviation
    real<lower=0, upper=1> theta1; // First threshold
    real<lower=0, upper=1> theta2; // Second threshold
}
model {
//prior distributions
    pho ~ beta(1,1);
    gamma ~ beta(1,1);
    for (i in 1:N) {
        if(is_correct[i]==1){
            is_confident[i] ~ bernoulli(pho);
            if(is_confident[i]==0){
                if(is_old[i]==1){
                    1~bernoulli(gamma);
                }
                else{0~bernoulli(gamma);}
            }
        }
        else{
            0~bernoulli(pho);
            if(is_old[i]==1){0~bernoulli(gamma);}
            else{1~bernoulli(gamma);}
        }
    }
"""

In [35]:
OneHT_model = stan.StanModel(model_code=OneHT_program_code)
TwoHT_model = stan.StanModel(model_code=OneHT_program_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1e7208daf67add1d04f08e2ac216baa9 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1e7208daf67add1d04f08e2ac216baa9 NOW.


In [32]:
##data preparation
data={
    'N':len(df),
    'is_old':df['old_new'],
    'is_confident':df['remember'],
    'is_correct': df['accuracy']
}

In [36]:
OneHT_fit = OneHT_model.sampling(data=data, chains=4, iter=4000,control={'adapt_delta': 0.99})
TwoHT_fit = TwoHT_model.sampling(data=data, chains=4, iter=4000,control={'adapt_delta': 0.99})

#convergence inspection and estimations
print(OneHT_fit)
print(TwoHT_fit)


Inference for Stan model: anon_model_1e7208daf67add1d04f08e2ac216baa9.
4 chains, each with iter=4000; warmup=2000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=8000.

        mean se_mean     sd    2.5%     25%     50%     75%   97.5%  n_eff   Rhat
mu     4.2e5   5.8e5  9.4e5  -5.3e5   -7923   2.6e4   1.7e5   3.2e6      3    3.2
sigma    inf     nan    inf 3.6e306 4.2e307 8.8e307 1.4e308 1.8e308    nan    nan
pho     0.84  3.4e-3    0.1     0.6    0.78    0.85    0.91    0.98    853    1.0
gamma   0.23  4.1e-3   0.12    0.06    0.14    0.21    0.31    0.49    817   1.01
lp__  695.24    0.06   1.47  691.54  694.49  695.57  696.35   697.1    663   1.01

Samples were drawn using NUTS at Thu Apr 11 23:29:35 2024.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
Inference for Stan model: anon_model_1e7208daf67add1d04f08e2ac216baa9.
4 chains, each with iter=